In [66]:

import selenium
import pandas  # 当前爬取部分pandas用于承接数据以获取更好的格式
import time
import datetime
import numpy
import random
import traceback
import threading
import queue


# 车型关系--待完善，后续待交路表部分完成后会使用对应映射而不是上面的简单类型映射
marshalling = {'CHR380BL': [], 'CHR380B': [], 'CHR380B重连': []}


species = {'新空调普快': ['120', 'LPPPPPP', 0], '新空调快速': ['120', 'LPPPPPP', 0],
           '新空调特快': ['140', 'LPPPPPP', 0], '新空调直快': ['160', 'LPPPPPP', 0],
           '动车': ['200', 'LPPL', 1], '城际': ['200', 'LPPL', 1], '高速': ['300', 'LPPLLPPL', 2]}
species1 = {'K': ['120', 'LPPPPPP', 0], 'T': ['140', 'LPPPPPP', 0], 'Z': ['160', 'LPPPPPP', 0],
            'D': ['200', 'LPPL', 1], 'C': ['200', 'LPPL', 1], 'G': ['300', 'LPPLLPPL', 2]}

station = {'武汉': ['a', -1, 0, 0, 0], '武汉动车所': ['c', 1, 0, 0, 20],
           '京广高速咸宁北方向': ['b', 1, 2, 1, 5], '武九客专葛店南方向': ['d', 1, 2, 1, 4],
           '京广高速孝感北方向': ['e', 1, 1, 2, 10], '沪蓉线红安西方向': ['f', 0, 1, 2, 10],
           '沪蓉线汉口方向': ['h', 0, 1, 2, 5], '丹水池联络线汉口方向': ['i', 0, 1, 2, 7], }

# 线路和车站关系，主要用于从车站-值获取线路-键
routewhchar = {'武汉动车所': ["武汉"],
               '京广高速咸宁北方向': ["咸宁北", "岳阳东", "长沙南"], '武九客专葛店南方向': ["葛店南", "鄂州", "黄冈西", "黄冈东", "黄梅东", "黄石北", "庐山"],
               '京广高速孝感北方向': ["孝感北", "信阳东", "漯河西", "驻马店东", "郑州东"], '沪蓉线红安西方向': ["红安西", "麻城北", "六安", "合肥", "合肥南"],
               '沪蓉线汉口方向': ["汉口"], '丹水池联络线汉口方向': ["汉口"], }
routewh1 = {'c': ["武汉"],
            'b': ["咸宁北", "岳阳东", "长沙南"], 'd': ["葛店南", "鄂州", "黄冈西", "黄冈东", "黄梅东", "黄石北", "庐山"],
            'e': ["孝感北", "信阳东", "漯河西", "驻马店东", "郑州东"], 'f': ["红安西", "麻城北", "六安", "合肥", "合肥南"],
            'f': ["汉口"], 'i': ["汉口"], }


In [67]:


def dealTime(datafr: pandas.DataFrame) -> pandas.DataFrame:
    # 对于始发车和终到车进行时间处理，默认相同
    for idx, row in datafr.iterrows():
        if row["开时"] == "----":
            row["开时"] = row["到时"]
        elif row["到时"] == "----":
            row["到时"] = row["开时"]
    # datafr['到时'].replace(to_replace="----", method="bfill", inplace=True)
    # datafr['开时'].replace(to_replace="----", method="ffill", inplace=True)

    # print(datafr)
    # 计算停站时间
    datafr["到时"] = pandas.to_datetime(datafr["到时"], errors="ignore")
    datafr["开时"] = pandas.to_datetime(datafr["开时"], errors="ignore")
    # 实发终到设置停站时间
    # pd["停站时间"].replace(0,10,inplace=True)

    return datafr


def stopStTime(stopTime: pandas.Timedelta) -> int:
    a = int(stopTime.seconds/60)  # 处理为数值类型的分钟时间
    # 始发或是终到的时间是随机的
    return a if a > 0 else random.randint(10, 30)


def ModtimeStr(switchTime: pandas.Timestamp) -> str:
    # 处理为仅有时分秒样式的字符串格式的时间
    return switchTime.strftime("%H:%M:%S")


def speMarType(traincode: str) -> str:  # 目前获取编组信息的方式
    til = species1[traincode[0]]  # 处理获取车辆信息字符串部分
    return "{cod} COMMUTER {speed} {mar} X1".format(cod=traincode, speed=til[0], mar=til[1])


def arrLeaTime(t1: pandas.Timestamp, st: list[str], mark: int) -> str:
    # 选取是上一站还是下一站
    tarst = st[mark]
    useTime = station[tarst][4]
    if mark == 0:  # 进场减时间
        res = t1-datetime.timedelta(minutes=useTime)
    else:  # 离场加时间
        res = t1+datetime.timedelta(minutes=useTime)
    # 返回时分秒格式的字符串
    return res.strftime("%H:%M:%S")

# 手动建立映射关系,别的方法不会


def getArrLeaSt(stype, station):
    stcode = ""
    # 多路径车站部分
    if stype == "高速":  # 高速
        ArrLeaveSt = {
            '汉口': '沪蓉线汉口方向',
            '宜昌东': '沪蓉线汉口方向',
            '襄阳东': '沪蓉线汉口方向',
            '成都东': '沪蓉线汉口方向',
            '利川': '沪蓉线汉口方向',
            '重庆北': '沪蓉线汉口方向',
        }
    else:  # 动车城际speedType == "动车" or speedType == "城际"
        ArrLeaveSt = {
            '汉口': '丹水池联络线汉口方向',
            '宜昌东': '丹水池联络线汉口方向',
            '襄阳东': '丹水池联络线汉口方向',
            '成都东': '丹水池联络线汉口方向',
            '利川': '丹水池联络线汉口方向',
            '重庆北': '丹水池联络线汉口方向',
        }

    stcode = ArrLeaveSt.get(station)
    if stcode != None:
        return stcode

    # 常规单路径车站
    ArrLeaveSt = {
        '武汉': '武汉动车所',
        '南宁东': '京广高速咸宁北方向',
        '北京西': '京广高速孝感北方向',
        '南昌西': '武九客专葛店南方向',
        '香港西九龙': '京广高速咸宁北方向',
        '广州南': '京广高速咸宁北方向',

        '深圳北': '武九客专葛店南方向',
        '西安北': '京广高速孝感北方向',
        '温州南': '武九客专葛店南方向',
        '兰州西': '京广高速孝感北方向',
        '长沙南': '京广高速咸宁北方向',
        '郑州东': '京广高速孝感北方向',
        '贵阳北': '京广高速咸宁北方向',
        '昆明南': '京广高速咸宁北方向',
        '上海虹桥': '沪蓉线红安西方向',
        '合肥东': '沪蓉线红安西方向',
        '厦门': '武九客专葛店南方向',
        '南京南': '沪蓉线红安西方向',
        '福州': '武九客专葛店南方向',
        '杭州东': '武九客专葛店南方向',
        '南昌': '武九客专葛店南方向',
        '济南': '京广高速孝感北方向',
        '青岛北': '京广高速孝感北方向',
        '沈阳北': '京广高速孝感北方向',
        '厦门北': '武九客专葛店南方向',
        '大冶北': '武九客专葛店南方向',
        '福州南': '武九客专葛店南方向',
        '黄冈东': '武九客专葛店南方向',
        '安庆': '武九客专葛店南方向',
    }
    stcode = ArrLeaveSt.get(station, "")
    if stcode == "":
        print(f"车站:{station} 未找到")
    return stcode


def checkin(entrance: str) -> str:
    # 终到车等不指定
    # print(entrance,end=" ")
    if entrance == '':
        return '0'
    # 检票口A5 非终到车
    try:  # 对于空缺导致的报错
        en = entrance.strip("检票口AB")
    except TypeError:
        return '0'
    else:
        return en


def routedispatch(x, mark=0) -> str:
    # 对于分场式车站按线路所分流进场车辆,目前仅设置进场线路所路由
    #print(stoptruck,"\t" ,routearr[mark])
    t = (x["到时"]-datetime.timedelta(minutes=2)).strftime("%H:%M:%S")
    stoptruck = x["检票口"]
    if stoptruck >= 1 and stoptruck <= 14:  # 高速场车辆
        k1 = {"武九客专葛店南方向": f" p#3#{t}#0",  # 武汉南线路所
              "沪蓉线汉口方向": f" n#2#{t}#0",  # 滠口线路所
              "丹水池联络线汉口方向": f" n#2#{t}#0"}
    else:  # 综合场车辆
        k1 = {
            "武九客专葛店南方向": f" p#1#{t}#0",
            "沪蓉线汉口方向": f" n#1#{t}#0",
            "丹水池联络线汉口方向": f" n#1#{t}#0"}

    return k1.get(x["前后站"][mark], "")


def tostrformat(row1):
    res = ""
    try:
        indexst = station.get(row1["车站名称"])
        res = " {0}#{1}#{2}#{3}".format(
            indexst[0], row1["股道"], row1["到达时间"], row1["停站时间"])
    except KeyError:
        print("KeyError in:", str(row1))
    finally:
        return res


In [68]:
# 手动建立映射关系,别的方法不会


def processTrains(routefile: str) -> pandas.DataFrame:
    # 处理产生初步所需信息的数据框，车次时刻表部分
    trainRouteFrame = pandas.read_csv(filepath_or_buffer=routefile,
                                      sep=",", encoding="utf8", header=0)
    # print(trainRouteFrame.head())
    trainRouteFrame.drop_duplicates(inplace=True)  # 常规处理
    trainRouteFrame.fillna(value="", inplace=True)
    trainRouteFrame = dealTime(trainRouteFrame)  # 初步处理时间
    trainRouteFrame.reset_index(drop=True, inplace=True)  # 重设索引

    trainRouteFrame["检票口"] = trainRouteFrame["检票口"].apply(lambda x: checkin(x))
    # 处理检票口和映射进场离场路线
    trainRouteFrame["arr"] = trainRouteFrame.apply(
        lambda x: getArrLeaSt(x["类型"], x["始发站"]), axis=1)
    trainRouteFrame["lea"] = trainRouteFrame.apply(
        lambda x: getArrLeaSt(x["类型"], x["终到站"]), axis=1)
    trainRouteFrame["前后站"] = trainRouteFrame.apply(
        lambda x: [x["arr"], x["lea"]], axis=1)  # 合并前后站

    return trainRouteFrame


In [69]:
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler, LabelEncoder
import matplotlib.pyplot as plt


mapdict = {'武汉': 0, '武汉动车所': 1,
           '京广高速咸宁北方向': 2, '武九客专葛店南方向': 3,
           '京广高速孝感北方向': 4, '沪蓉线红安西方向': 5,
           '沪蓉线汉口方向': 6, '丹水池联络线汉口方向': 7, }


def processNoEntry(datafr: pandas.DataFrame):
    # le = LabelEncoder()  # 数值化处理映射字符类型为数值
    # for col in ["类型", "arr", "lea"]:
    #     datafr[col] = le.fit_transform(datafr[col])
    datafr["类型"] = datafr["类型"].map({"动车": 0, "高速": 1})

    datafr["arr"] = datafr["arr"].map(mapdict)
    datafr["lea"] = datafr["lea"].map(mapdict)

    # 数值化处理时间类型，转化为数值时间戳,归一化无所谓
    datafr["开时"] = datafr["开时"].astype('int64')
    datafr["到时"] = datafr["到时"].astype('int64')

    datafr["检票口"] = datafr["检票口"].map(float)
    # print(datafr.corr())
    # 取需要的特征值
    dataNumic = datafr.drop(columns=["车次",  "始发站", "终到站", "前后站"])
    # 训练用的样本，各项数值均正常
    X_d = dataNumic[dataNumic["检票口"] != 0].drop(columns=["检票口"])
    y = dataNumic[dataNumic["检票口"] != 0]["检票口"]
    # 最终需要预测的空值，返回预测的结果
    X_test = dataNumic[dataNumic["检票口"] == 0].drop(columns=["检票口"])
    # 决策树部分，对于分支和叶节点均给出相对较大的数字
    # dts = DecisionTreeClassifier(
    #     max_depth=10, min_samples_leaf=3, min_samples_split=6)
    dts = RandomForestClassifier(n_estimators=8, random_state=0,
                                 max_depth=10, min_samples_leaf=3, min_samples_split=6)

    dts.fit(X_d, y)
    pre = dts.predict(X_test)  # 预测结果
    print("训练集上准确率: ", dts.score(X_d, y))  # 正确率
    # fig = plt.figure(figsize=(100, 50))  # 可视化决策树图?
    # plot_tree(decision_tree=dts, feature_names=[
    #     "arrivetime", "leavetime", "arrst", "least", "ckeckin"], max_depth=5, fontsize=10, filled=True)
    # fig.savefig(fname="f1.png")

    val, con = numpy.unique(ar=pre, return_counts=True)
    print(dict(zip(val, con)))
    return pre

#at = processTrains("wh1.csv")
# processNoEntry(at)


In [70]:

from pyecharts.charts import Line, Bar, Sankey, Timeline
from pyecharts import options as opts
import datetime

maplist = ['武汉动车所',
           '京广高速咸宁北方向', '武九客专葛店南方向',
           '京广高速孝感北方向', '沪蓉线红安西方向',
           '沪蓉线汉口方向', '丹水池联络线汉口方向']

datafr = processTrains("wh1.csv")


def timedata(datafr: pandas.DataFrame, station: str, arrlea: str) -> list:
    datafr["小时"] = datafr["到时"].apply(lambda x: x.hour)  # 分组时间列
    # 按照车站和小时分组，小时用于保留数据便于后续操作
    datafrgroup = datafr.groupby(by=[arrlea, "小时"]).count()
    datafrgroup.reset_index(level="小时", inplace=True)  # 解开时间索引便于后续获取数据

    sttimedata = datafrgroup.loc[station]  # 按照车站获取数据

    datares = [0]*24  # 承接返回值，24个0防止溢出

    tl = sttimedata["小时"].to_list()  # 时间序列，只使用第一个
    cl = sttimedata["车次"].to_list()  # 车次分布序列，用于画图映射

    datares[tl[0]:len(tl)] = cl  # 覆盖数据为车次分布
    return datares


def timeplot(datafr: pandas.DataFrame, al: str, ti: str):
    line = Line()  # 创建折线图实例

    time_data = [datetime.datetime(2023, 11, 3, i) for i in range(0, 23)]
    line.add_xaxis(time_data)  # 添加时间轴数据

    for st in maplist:  # 添加 y 轴数据
        line.add_yaxis(series_name=st, y_axis=timedata(datafr, st, al))
    # 设置时间轴等全局设置
    line.set_global_opts(xaxis_opts=opts.AxisOpts(splitline_opts=opts.SplitLineOpts(is_show=True),
                                                  type_='time', axislabel_opts=opts.LabelOpts(is_show=True, rotate=-45,)),
                         yaxis_opts=opts.AxisOpts(
                             splitline_opts=opts.SplitLineOpts(is_show=True)),
                         title_opts=opts.TitleOpts(
                             title=ti, subtitle="数量时间变化图"),
                         legend_opts=opts.LegendOpts(pos_bottom='50%', pos_left=0, orient="vertical"))

    return line  # 渲染图表


def sankeypolt(datafr: pandas.DataFrame):
    tdarr = []
    tdlea=[]
    for st in maplist:
        tdarr.append(timedata(datafr, st, "arr"))
        tdlea.append(timedata(datafr, st, "lea"))

    skdarr = pandas.DataFrame(data=tdarr, index=maplist)
    skdataarr = skdarr.transpose().head(n=24)

    skdlea = pandas.DataFrame(data=tdlea, index=maplist)
    skddatalea = skdlea.transpose().head(n=24)
   
    tl = Timeline()
    for i in range(0, 24):
        rowlistarr = skdataarr.loc[i].to_list()

        link1 = [{"source": skdataarr.columns[j]+"进场", "target": "武汉站",
                  "value": int(rowlistarr[j])} for j in range(0, len(rowlistarr)-1)]
        
        rowlistlea = skddatalea.loc[i].to_list()
        link2 = [{"source": "武汉站", "target":skddatalea.columns[j]+"离场" ,
                  "value": int(rowlistlea[j])} for j in range(0, len(rowlistlea)-1)]
        
        node1=[{"name":skdataarr.columns[j]+"进场" } for j in range(0, len(rowlistarr)-1)]
        node2=[{"name":skdataarr.columns[j]+"离场" } for j in range(0, len(rowlistarr)-1)]

        node1.append({"name":"武汉站"})

        sk = Sankey()
        sk.add(
            series_name="sankey",
            nodes=node1+node2,
            links=link1+link2,
            linestyle_opt=opts.LineStyleOpts(curve=0.5,opacity=2),
            label_opts=opts.LabelOpts(position="bottom",font_size=10),
        )
        sk.set_global_opts(title_opts=opts.TitleOpts(title="Sankey-基本示例"))
        sk.render_notebook()
        tl.add(chart=sk, time_point="{0}时".format(i))
    return tl


s = sankeypolt(datafr)
s.render_notebook()
#s.render("sank1.html")


C:\Users\z'z'x\AppData\Local\Temp\ipykernel_27032\2484381920.py:13: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  datafr["到时"] = pandas.to_datetime(datafr["到时"], errors="ignore")
C:\Users\z'z'x\AppData\Local\Temp\ipykernel_27032\2484381920.py:14: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  datafr["开时"] = pandas.to_datetime(datafr["开时"], errors="ignore")


In [34]:

timeplot(datafr, "arr", "武汉站列车到达情况").render_notebook()


In [35]:
timeplot(datafr, "lea", "武汉站列车出发情况").render_notebook()
